In [1]:
import Vid_to_Strain as predict
import loader

In [2]:
root = loader.dataModules()
root

'/Users/ishan/Box/Strain Study'

In [3]:
from tqdm import tqdm
import os

dataset_run = "Netherlands2"
flip_condition = False

netherlands_dataset = os.path.join(root, "Netherlands") #2,534 videos
netherlands2_dataset = os.path.join(root, "Netherlands2", "LV_preprocessed") #749 videos
a4c_dataset = os.path.join(root, "StrainStudyA4c-Preprocessed") #167 videos

weights_path = os.path.join(root, "Weights-20201103T193519Z-001", "Weights", "deeplabv3_resnet50_random.pt")
segmentations_path = os.path.join(root, "Ishan", "FirstCodebase", "Netherlands2", "segmentations")
strains_path = os.path.join(root, "Ishan", "FirstCodebase", "Netherlands2", "strains")
plot_path = os.path.join(root, "Ishan", "FirstCodebase", "Netherlands2", "plot")
excel_path = os.path.join(root, "Ishan", "FirstCodebase", "Netherlands2", "excel")

# Choosing dataset to run
if dataset_run == "Netherlands":
    dataset = netherlands_dataset
    flip_condition = True
elif dataset_run == "Netherlands2":
    dataset = netherlands2_dataset
    flip_condition = True
else:
    dataset = a4c_dataset
    flip_condition = False

count = 0
# Loop to run script on dataset
for video in tqdm(os.listdir(dataset)):
    videoSplit = os.path.splitext(video)
    extensionName = videoSplit[1]
    
    count += 1
    
    if extensionName == ".avi":# and count > 87:
        videoPath = os.path.join(dataset, video)
        
        try:
            predict.estimate_strain(videoPath, flip=flip_condition, weights=weights_path, segmentation_dir=segmentations_path,
                                strain_dir=strains_path, plot_dir=plot_path, excel_dir=excel_path, dilations=1, point_radius=1)
        except:
            break

  0%|          | 3/749 [03:28<14:25:32, 69.61s/it]


<Figure size 432x288 with 0 Axes>